# Workflow with artifacts

This Notebook aim to show how data can be downloaded from Minio.
This notebook can be launched only after the upload of some data in Notebook 1 (Minio example).
The task will be responsible to list files in root directory and in the directory downloaded from Minio.

Notice that Minio tar.gz file is decompressed automatically

In [27]:
from argo.workflows.client import V1alpha1Api
from argo.workflows.config import load_kube_config

In [28]:
load_kube_config()  # loads local configuration from ~/.kube/config

In [29]:
v1alpha1 = V1alpha1Api()

You need to create credentials for minio storage:

```
kubectl create secret generic s3-secret --from-literal=accessKey=minioadmin --from-literal=secretKey=minioadmin
```

In [30]:
from argo.workflows.client.models import *

In [24]:
from kubernetes.client.models import V1SecretKeySelector
access_key_secret = V1SecretKeySelector(name="s3-secret", key="accessKey")
secret_key_secret = V1SecretKeySelector(name="s3-secret", key="secretKey")

s3_artifact = V1alpha1Artifact(name="s3-artifact", 
                               path="subjects/0040013",
                               s3=V1alpha1S3Artifact(bucket="subjects-data",
                                       endpoint="minio.default:9000",
                                       insecure=True,
                                       access_key_secret=access_key_secret,
                                       secret_key_secret=secret_key_secret,
                                                     key="controllo-0040013.tar.gz"))

In [31]:
# Create a Workflow with a single task

inputs = V1alpha1Inputs(artifacts=[s3_artifact])

spec = V1alpha1WorkflowSpec(
    entrypoint="catfile", 
    templates=[
        V1alpha1Template(
            name="catfile", 
            container=V1Container(
                name="dumper", 
                image="ubuntu", 
                command=["bash", "-c", "cd / ; ls -lh ; cd subjects/0040013 ; ls -lh"], 
                args=[]), 
            inputs=inputs)
    ])
        
workflow = V1alpha1Workflow(
    api_version="argoproj.io/v1alpha1",
    kind="Workflow",
    metadata=V1ObjectMeta(generate_name="cat-file-"), 
    spec=spec,
    status=V1alpha1WorkflowStatus())

In [32]:
result = v1alpha1.create_namespaced_workflow('default', workflow)

In [37]:
# The following fragment must be invoked more time, until the workflow is completed its execution

from kubernetes.client.rest import ApiException
from kubernetes import client, config

config.load_kube_config()

wfs = v1alpha1.list_namespaced_workflows(namespace="default")
for job in wfs.items:
    if job.metadata.name == result.metadata.name:
        for pod_name in job.status.nodes.keys():
            if job.status.nodes[pod_name].type == 'Pod':
                try:
                    api_instance = client.CoreV1Api()
                    # Pod created by Argo has init containers, main container and wait container...
                    api_response = api_instance.read_namespaced_pod_log(name=pod_name, container="main", namespace='default')
                    print(f"Log for {pod_name}: {api_response}")
                except ApiException as e:
                    print(f'Found exception in reading the logs {pod_name}')

Log for cat-file-9qpcx: total 68K
drwxr-xr-x   2 root root 4.0K Dec  2 12:43 bin
drwxr-xr-x   2 root root 4.0K Apr 24  2018 boot
drwxr-xr-x   5 root root  360 Dec 27 16:09 dev
drwxr-xr-x   1 root root 4.0K Dec 27 16:09 etc
drwxr-xr-x   2 root root 4.0K Apr 24  2018 home
drwxr-xr-x   8 root root 4.0K May 23  2017 lib
drwxr-xr-x   2 root root 4.0K Dec  2 12:43 lib64
drwxr-xr-x   2 root root 4.0K Dec  2 12:43 media
drwxr-xr-x   2 root root 4.0K Dec  2 12:43 mnt
drwxr-xr-x   2 root root 4.0K Dec  2 12:43 opt
dr-xr-xr-x 525 root root    0 Dec 27 16:09 proc
drwx------   2 root root 4.0K Dec  2 12:43 root
drwxr-xr-x   1 root root 4.0K Dec 27 16:09 run
drwxr-xr-x   1 root root 4.0K Dec 19 04:21 sbin
drwxr-xr-x   2 root root 4.0K Dec  2 12:43 srv
drwxr-xr-x   3 root root 4.0K Dec 27 16:09 subjects
dr-xr-xr-x  13 root root    0 Dec 26 22:48 sys
drwxrwxrwt   2 root root 4.0K Dec  2 12:43 tmp
drwxr-xr-x   1 root root 4.0K Dec  2 12:43 usr
drwxr-xr-x   1 root root 4.0K Dec  2 12:43 var
total 8.0K
d